In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from time import time
from collections import Counter

import spacy



In [2]:
df2=pd.read_csv(r'C:\Users\pnl0gp8t\Downloads\simpsons_script_lines\simpsons_script_lines.csv')

C:\Users\pnl0gp8t\AppData\Local\Temp\ipykernel_9764\2258381421.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv(r'C:\Users\pnl0gp8t\Downloads\simpsons_script_lines\simpsons_script_lines.csv')


In [3]:
df2.shape

(158271, 13)

In [4]:
df2.head(2)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3


In [5]:
df2['raw_text'].iloc[900]

'Homer Simpson: "Better half," Marge. "Better half."'

In [6]:
df=df2[['raw_character_text','spoken_words']]

In [18]:
df.shape

(132110, 2)

In [19]:
df.head(2)

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?


In [20]:
df.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

In [21]:
df=df.dropna().reset_index(drop=True)
df.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

In [26]:
nlp=spacy.load('en_core_web_sm',disable=['ner','pasrser'])
def cleaning(docs):
    txt=[tokens.lemma_ for tokens in docs if not tokens.is_stop]
    if len(txt)>2:
        return " ".join(txt)
    

In [27]:
brief_cleaning = (re.sub("[^A-Za-x]+", ' ', str(row)).lower() for row in df['spoken_words'])

In [28]:
brief_cleaning

<generator object <genexpr> at 0x00000153F2039350>

In [29]:
import re
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 3.57 mins


In [30]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(96645, 1)

In [31]:
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in df_clean['clean']]

In [32]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [37]:
from collections import defaultdict 
bigram = Phraser(phrases)
sentences = bigram[sent]
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

29036

In [38]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['ou', 's', 'm', 'oh', 'don_t', 'like', 'll', 'know', 'think', 'thing']

In [39]:
import multiprocessing

from gensim.models import Word2Vec

In [40]:
cores = multiprocessing.cpu_count()

In [41]:
cores

8

In [42]:
wv_model=Word2Vec(
    min_count=20,
    window=2,
    sample=6e-5,
    min_alpha=0.0007,
    negative=20,
    workers=cores-1
)

In [43]:
wv_model.build_vocab(sentences,progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 9.94 mins


In [44]:
t=time()
wv_model.train(sentences,total_examples=wv_model.corpus_count,epochs=30,report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 1.34 mins


In [45]:
wv_model.init_sims(replace=True)

C:\Users\pnl0gp8t\AppData\Local\Temp\ipykernel_9764\1461675014.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  wv_model.init_sims(replace=True)


In [90]:
# Approach:¶
# Parse the text from the body of each document using Natural Language Processing (NLP).
# Turn each document instance di into a feature vector Xi using Term Frequency–Inverse Document Frequency (TF-IDF).
# Apply Dimensionality Reduction to each feature vector Xi using t-Distributed Stochastic Neighbor Embedding (t-SNE) to cluster similar research articles in the two-dimensional plane X embedding Y1.
# Use Principal Component Analysis (PCA) to project down the dimensions of X to several dimensions that will keep .95 variance while removing noise and outliers in embedding Y2.
# Apply k-means clustering on Y2, where k is 20, to label each cluster on Y1.
# Apply Topic Modeling on X using Latent Dirichlet Allocation (LDA) to discover keywords from each cluster.
# Investigate the clusters visually on the plot, zooming down to specific articles as needed, and via classification using Stochastic Gradient Descent (SGD).

In [91]:
# Table of Contents
# Loading the data
# Pre-processing
# Vectorization
# PCA & Clustering
# Dimensionality Reduction with t-SNE
# Topic Modeling on Each Cluster
# Classify
# Plot
# How to Use the Plot?
# Conclusion
# Citation/Sources

In [47]:
wv_model.wv.most_similar(positive=["terrific"])

[('script', 0.7897329330444336),
 ('worry', 0.7718368768692017),
 ('comment', 0.7688713073730469),
 ('badl', 0.7685776352882385),
 ('thinking', 0.7652694582939148),
 ('suspicious', 0.7628375291824341),
 ('imagination', 0.7618607878684998),
 ('necessar', 0.761447548866272),
 ('obviousl', 0.7605483531951904),
 ('decision', 0.7577673196792603)]

In [48]:
wv_model.wv.most_similar(positive=["congratulation"])

[('select', 0.7624908089637756),
 ('debate', 0.7137084007263184),
 ('governor', 0.7099790573120117),
 ('annual', 0.709835410118103),
 ('aboard', 0.7043335437774658),
 ('propose', 0.6971352100372314),
 ('cheech', 0.6910132169723511),
 ('essa', 0.6879330277442932),
 ('power_plant', 0.6869155764579773),
 ('transfer', 0.6853348016738892)]

In [49]:
wv_model.wv.most_similar(positive=["marge"])

[('homer', 0.7343037724494934),
 ('homie', 0.6990343332290649),
 ('reverend_lovejo', 0.6899734735488892),
 ('ned', 0.6786044836044312),
 ('sorr', 0.6782078742980957),
 ('embarrassed', 0.6770483255386353),
 ('comfort', 0.6768039464950562),
 ('rude', 0.6710934638977051),
 ('spoil', 0.6708145141601562),
 ('balone', 0.670415997505188)]

In [52]:
wv_model.wv.most_similar(positive=["homer_simpson"])

[('governor', 0.7033300995826721),
 ('burns', 0.6972688436508179),
 ('recent', 0.6850525140762329),
 ('pleased', 0.677588164806366),
 ('springfielder', 0.6677001714706421),
 ('speaker', 0.6673247814178467),
 ('easil', 0.6573867797851562),
 ('gem', 0.6498971581459045),
 ('montgomer_burn', 0.6460747718811035),
 ('united_states', 0.6459463834762573)]

In [56]:
wv_model.wv.similarity("maggie", 'baby')

0.44911608

In [58]:
wv_model.wv.similarity('bart', 'nelson')

0.57744515

In [59]:
wv_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

[('man', 0.5958160161972046),
 ('married', 0.5842220187187195),
 ('attractive', 0.5816301107406616)]

In [60]:

wv_model.wv.doesnt_match(["nelson", "bart", "milhouse"])

'nelson'

In [61]:

wv_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

[('lisa', 0.6889214515686035),
 ('surprised', 0.6846040487289429),
 ('mom_dad', 0.6530520915985107)]

In [62]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [63]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=50).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))
    

In [67]:
tsnescatterplot(wv_model, 'maggie', [i[0] for i in wv_model.wv.most_similar(negative=["maggie"])])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 300 and the array at index 1 has size 100